
<center>
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> Tarea N°1, v1.00 </h1>
    
</center>

<p>

</p>

## Instrucciones

Felipe Montero Concha 201473611-8

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 30 de Abril a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).




## Introducción 

En esta primera tarea de INF/ILI-285, versión 2020-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Significancia, Errores de Cancelación y Búsqueda de Ceros. El desarrollo de cada uno de esos temas se presenta en una serie de preguntas, donde ustedes deben ir decidiendo, pregunta a pregunta, como cada uno de los temas se aplica. En general, los temas no se analizan por separado, sino de manera acoplada. Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.

## Problemas

In [1]:
import numpy as np
import bitstring
import scipy.special

### 1. Simulador (50 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}


Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$) y la cantidad de bits que tiene en la mantisa un computador ficticio, el manejo del exponente debe considerarse de las misma forma que lo maneja _double precision_.
Esta implementación debe calcular las raíces de $f$ sin perdida de significancia con la cantidad de bits disponibles para la mantisa.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

Considere que en ese computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces.  Considere el siguiente ejemplo:

```python
# Alg. Base
a = 9819824.624837
b = 148736.523476
c = a+b

# Alg. con Representación de Punto Flotante de 'bits_mant' bits en la mantisa.
am = f_new_rep(9819824.624837,bits_mant) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant) # Aproximar el input en la nueva representación.
cm = f_new_rep(m,exp,am+bm) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [ ]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
output
eval      : (double) resultado obtenido
"""
#Esta funcion lo unico que hace es sumar el 1 en caso de que sea necesario a un numero binario, 
#acarreando el 1 cuando es necesario
def sumar_1_bin(num):
    a_sumar = str(num)[::-1]
    result = ""
    debe = True
    for bit in a_sumar:
        if bit == "1" and debe:
            result += "0"
        elif bit == "0" and debe:
            result += "1"
            debe = False
        else:
            result += bit
    return result[::-1]
        

def f_new_rep(x,bits_mant):
  # Algoritmo de representación de punto flotante modificada.
    rep_IEEE = bitstring.BitArray(float= x, length = 64)
    ceros_unos = rep_IEEE.bin

    signo = ceros_unos[0]
    exponente = ceros_unos[1:12]
    mantisa = ceros_unos[12:]
    
    new_mantisa = mantisa[0:bits_mant]
    check_mant = mantisa[bits_mant:53]
    
    if check_mant[0] == "0":
        new_mantisa = new_mantisa
    elif check_mant[0] == "1":
        if "1" in check_mant[1:]:
            new_mantisa = sumar_1_bin(new_mantisa)
        elif "1" not in check_mant[1:]:
            if new_mantisa[-1] == "1":
                new_mantisa = sumar_1_bin(new_mantisa)
    dec_exponente = int(exponente,2)-1023
    total = 2**dec_exponente
    k = dec_exponente - 1  
    for i in range(0,bits_mant):
        total += int(new_mantisa[i])*(2**(k))
        k-=1
    if int(signo):
        total = total*-1
    return  total





print(f_new_rep(90,32))


In [ ]:
#Para todo polinomio cubico de esta caracteristica, el 1 siempre es raiz por lo tanto se procede a hacer ruffini en el codigo
# dividiendo el polinomio en 1, obteniendo los coeficientes de la cuadratica correspondiente para luego obtener a traves de
#la formula de soluciones cuadraticas el resto de las raices del polinomio cubico original.
def f_find_roots(A,bits_mant):
    
    coeficientes = [1,-A,A,-1]
    c_cuadraticos = [coeficientes[0]]
    for i in range(0,len(coeficientes)):
        if i == 0:
            a_sumar = coeficientes[0]*1
        else:
            c_cuadraticos.append(coeficientes[i]+a_sumar)
            a_sumar = coeficientes[i] + a_sumar
    print(c_cuadraticos)
    
    a = f_new_rep(c_cuadraticos[0],bits_mant) 
    b = f_new_rep(c_cuadraticos[1],bits_mant) 
    c = f_new_rep(c_cuadraticos[2],bits_mant)
    
    b_cuadrado = f_new_rep(b**2,bits_mant)
    dos_a = f_new_rep(2*a,bits_mant)
    ace = f_new_rep(4*a*c,bits_mant)
    
    resta_raiz = f_new_rep(b_cuadrado-ace,bits_mant)
    
    
    raiz = f_new_rep(np.sqrt(resta_raiz),bits_mant)
    
    
    sum1 = f_new_rep(-b + raiz , bits_mant)
    sum2 = f_new_rep(-b - raiz , bits_mant)
    
    x1 = f_new_rep(sum1/dos_a,bits_mant)
    x2 = f_new_rep(sum2/dos_a,bits_mant)
    
    x_roots = (1,x1,x2)
    return x_roots

f_find_roots(90,32)

### Polinomios de Legendre (50 puntos)
Dada la función compuesta $f$
\begin{equation}
f_{n,m}(x) = L_n(C_m(x)),
\end{equation}
donde $L_n$ es conocido como el polinomio de Legendre de grado $n$ definido de la siguiente forma:
\begin{equation}
L_{n}(x)=\frac{1}{2^{n}} \sum_{k=0}^{n}\left(\begin{array}{l}
n \\
k
\end{array}\right)^{2}(x-1)^{n-k}(x+1)^{k},
\end{equation}
y $C_m$ es el polinomio de Chebyshev
\begin{equation}
C_m(x) = \cos(m \cdot \arccos(x)).
\end{equation}

Utilizando el método de Bisección y Punto fijo se pide obtener la raíz de $f$ más cercana a $0.5$ dado un valor de $m$ y $n$



*Hint: Las raíces de Legendre son conocidas*



In [3]:
#Por bisección se procede conociendo que el chebyshev de grado n se comporta de forma muy similar al legendre de grado n
# por lo tanto se le realiza una bisección en base a una raiz encontrada al chebyshev de grado n , la cual actua como 
# x_0 para la función utilizada a comparar con el chebyshev de grado m
"""
input
n: (int) grado del polinomio de Legendre
m: (int) grado del polinomio de Chebyshev
tol: (double) tolerancia para la detención del algoritmo
output
root: (double) raiz obtenida
"""
def f_Biseccion(n, m, tol):
    i = 0
    menor_dif = float("inf")
    while i < n:
            valor_a_buscar = np.cos((np.pi*((2*i)-1))/(2*n))
            if abs(valor_a_buscar-0.5) < menor_dif:
                elegido = valor_a_buscar
                menor_dif = abs(valor_a_buscar-0.5)
            i+=1
    
    b = 1
    a = np.cos((np.pi/n)*(n-2))
    
    funcioncita = lambda x : elegido - np.cos(m*np.arccos(x))
 

    i = 0
    c = 0
    anterior = float("inf")
    
    while (abs(a-b) > tol) and (anterior != c):
        anterior = c
        c = (b+a)/2
        if funcioncita(c) == 0:
            return c
        else:
            if funcioncita(a)*funcioncita(c) < 0:
                b = c
            else:
                a = c
        i+=1

    x_m = np.cos(m*np.arccos(c))
    coef = scipy.special.legendre(n)
    print(sum(scipy.special.eval_legendre(coef,x_m)))
    return x_m
"""
def f_FPI(n, m, tol):
    x =  x_0 
    iteraciones=[]
    errores = []
    i = 0
    while abs(x_calc-g(x)) < tol:
        try:
            x_calc = g(x)
        except OverflowError:
            return None
        if x_calc == x and (x_calc not in iteraciones):
            iteraciones.append(x_calc)
        elif x_calc == x and (x_calc in iteraciones):
            i = n_iter
        else:
            iteraciones.append(x_calc)
        x = x_calc
        i+=1
    
 
    return root
"""
f_Biseccion(4,2,1e-8)

2.4509337506865823


0.38268345059581965

###  Determinantes (20 puntos)

Dada una matriz de dimensiones $ n \times n$ de la forma:
\begin{equation}
A
=
\begin{pmatrix}
a_{1,1} & a_{1,2}  & \dots & a_{1,n} \\
a_{2,1} & a_{2,2}  & \dots & a_{2,n} \\
\vdots &  \vdots   & \ddots & \vdots \\
a_{n,1} & a_{n,2}  & \dots & a_{n,n}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
donde $\mathbf{r}_k$ representa la $k$-ésima fila de la matriz $A$.
Considere la siguiente matriz $C_{i,j}(\alpha)$,
\begin{equation}
C_{i,j}(\alpha)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_i\,(1-\alpha)+\mathbf{r}_j\,\alpha \\
\vdots \\
\mathbf{r}_j\,(1-\alpha)+\mathbf{r}_i\,\alpha \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
de lo cual sabemos que $C_{i,j}(0)=A$ y que $C_{i,j}(1)$ es la matriz $A$ donde se intercambiaron las filas $i$ y $j$, es decir:
\begin{equation}
C_{i,j}(1)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_j \\
\vdots \\
\mathbf{r}_i \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix}.
\end{equation}
De las relaciones anteriores podemos concluir que el determinante de la matriz $A$, denominado $D=\det(A)$, es igual al determinante de $C_{i,j}(0)$, es decir $\det(C_{i,j}(0))=\det(A)=D$.
Por el otro lado, el determinante de $C_{i,j}(1)$ es $-D$, dado que es el intercambio de las filas $i$ y $j$ de la matriz $A$.
Por lo cual podemos concluir que $-D\leq \det(C_{i,j}(\alpha))\leq D$.

Utilizando el método de Bisección debe encontrar el valor de $\alpha$ con $p$ decimales de precisión que permitan que, dado los índices de las filas $i$, $j$, y $i\neq j$, el determinante de la matriz sea igual a $d$, donde $d\in[-D,D]$. 

Para esto se debe implementar una función que reciba la matriz $A$, las filas $i$ y $j$, y $p$; y retorne $\widehat{\alpha}$ tal que $\det(C_{i,j}(\widehat{\alpha}))=d$.


In [ ]:
#Aca se determino que hay que calcular la biusección a parti de la función d - det(M(alpha))
# el intervalo de itaración de alpha es entre 0 y 1 ya que no puede salir de esos valores, considerando que
#los valores alpha y 1-alpha estan relacionados.
"""
input
A: (array n x n) matriz
i: (int) índice de la fila "i".
j: (int) índice de la fila "j".
p: (int) cantidad de decimales de precision 
d: (double) valor requerido del determinante de $C_{i,j}(\alpha)$, i.e. $\det(C_{i,j}(\widehat{\alpha}))=d$.
output
alpha_hat: (double) alpha_hat tal que det(C_{i,j}(alpha_hat))=d.
"""
def generate_c(A,i,j,alpha):
    a_aux = np.array(A)
    fila_i = np.array(A[i])
    fila_j = np.array(A[j])
    aux = 0
    filas = []
    for x in range(0,len(A)):
        if x == i:
            aux = fila_i*alpha + fila_j*(1.-alpha)
            filas.append(aux)
        elif x == j:
            aux = fila_i*alpha + fila_j*(1.-alpha)
            filas.append(aux)
        else:
            filas.append(a_aux[x])
    print("este es el alfa de la funcion"+str(alpha))
    retorno = np.array(filas)
    print(retorno)
    return retorno



def find_alpha_hat(A,i,j,p,d):
    tolerancia =10**(-p-1)
    
    funcioncita = lambda C,i,j,alpha_i : d - np.linalg.det(generate_c(C,i,j,alpha_i))
    a = 0
    b = 1
    anterior = float("inf")
    print(generate_c(A,i,j,0.75))

    while (abs(a-b) > tolerancia) :
        c = (b+a)/2
        if funcioncita(A,i,j,c) == 0:
            alpha_hat = c
            return alpha_hat
        else:
            if funcioncita(A,i,j,a)*funcioncita(A,i,j,c) < 0:
                b = c
            else:
                a = c
    alpha_hat = c
    print(np.linalg.det(generate_c(A,i,j,alpha_hat)))
    return alpha_hat


A = np.array([[19,12,33,9],[3,4,6,4],[7,8,9,2],[5,9,6,1]])
print(np.linalg.det(A))
print(find_alpha_hat(A,0,2,7,4))

# Referencias

Si corresponde

In [1]:
def f_new_rep(x,bits_mant):
  # Algoritmo de representación de punto flotante modificada.
    rep_IEEE = bitstring.BitArray(float= x, length = 64)
    ceros_unos = rep_IEEE.bin

    signo = ceros_unos[0]
    exponente = ceros_unos[1:12]
    mantisa = ceros_unos[12:]
    
    new_mantisa = mantisa[0:bits_mant]
    check_mant = mantisa[bits_mant:53]
    
    if check_mant[0] == "0":
        new_mantisa = new_mantisa
    elif check_mant[0] == "1":
        if "1" in check_mant[1:]:
            new_mantisa = sumar_1_bin(new_mantisa)
        elif "1" not in check_mant[1:]:
            if new_mantisa[-1] == "1":
                new_mantisa = sumar_1_bin(new_mantisa)
    dec_exponente = int(exponente,2)-1023
    total = 2**dec_exponente
    k = dec_exponente - 1  
    for i in range(0,bits_mant):
        total += int(new_mantisa[i])*(2**(k))
        k-=1
    if int(signo):
        total = total*-1
    return  total

def f_new_rep(x,bits_mant):
  # Algoritmo de representación de punto flotante modificada.
    rep_IEEE = bitstring.BitArray(float= x, length = 64)
    ceros_unos = rep_IEEE.bin

    signo = ceros_unos[0]
    exponente = ceros_unos[1:12]
    mantisa = ceros_unos[12:]
    
    new_mantisa = mantisa[0:bits_mant]
    check_mant = mantisa[bits_mant:53]
    
    if check_mant[0] == "0":
        new_mantisa = new_mantisa
    elif check_mant[0] == "1":
        if "1" in check_mant[1:]:
            new_mantisa = sumar_1_bin(new_mantisa)
        elif "1" not in check_mant[1:]:
            if new_mantisa[-1] == "1":
                new_mantisa = sumar_1_bin(new_mantisa)
    print(exponente)
    print(new_mantisa)
    return  total

